### Requirements

In [5]:
import pandas as pd
import requests
import json

# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# from bs4 import BeautifulSoup

### Scraping from API: https://newsapi.org/v2/

In [7]:
f = open('../raw_data/news_api/sources.json',) 
# returns JSON object as a dictionary
data = json.load(f)
f.close()
sources_df = pd.DataFrame(data["sources"])

In [9]:
files_list = []
number_of_requests = 5
api = "940f51da7bc0439b8cf24417ae0bd5e4" # API do Martins

# the maximum number of news articles per request is 100
# the maximum number of requests per day is 100
for number in range(number_of_requests):
    sources_df = sources_df[sources_df["language"] == "en"].sample(20) # grabs only 20 sources (max size) at random each time these are selected
    sources_list = sources_df.id.to_list()
    
    sources = ",".join(sources_list)
    url = 'https://newsapi.org/v2/'
    url_params = 'everything' #everything or top headlines
    params = { "sources": sources,
            "pageSize": 100,
            'apiKey': api}

    response = requests.get(url + url_params, params=params)
    
    file = response.json()["articles"]
    files_list.append(file)

### Putting result into dataframe

In [16]:
elems =[]
for file in files_list:
    for elem in file:
        elems.append(elem)
news_api_df = pd.DataFrame(elems)

## Alternative way
# files_list = [file1, file2]
# files_list_unpacked = [*file1, *file2]
# news_api_df = pd.DataFrame([*file1, *file2])

### Info do scaping do José (em baixo)

In [6]:
my_dict = {'CRIME': 'Crime',
 'ENTERTAINMENT': 'Entertainment',
 'WORLD NEWS': 'World News',
 'IMPACT': 'Other',
 'POLITICS': 'Politics',
 'WEIRD NEWS': 'Other',
 'BLACK VOICES': 'Activism',
 'WOMEN': 'Entertainment',
 'COMEDY': 'Entertainment',
 'QUEER VOICES': 'Activism',
 'SPORTS': 'Sports',
 'BUSINESS': 'Business',
 'TRAVEL': 'Culture',
 'MEDIA': 'Media',
 'TECH': 'Technology',
 'RELIGION': 'Religion',
 'SCIENCE': 'Science',
 'LATINO VOICES': 'Activism',
 'EDUCATION': 'Education',
 'COLLEGE': 'Education',
 'PARENTS': 'Other',
 'ARTS & CULTURE': 'Culture',
 'STYLE': 'Trends',
 'GREEN': 'Activism',
 'TASTE': 'Culture',
 'HEALTHY LIVING': 'Health',
 'THE WORLDPOST': 'World News',
 'GOOD NEWS': 'Other',
 'WORLDPOST': 'World News',
 'FIFTY': 'Other',
 'ARTS': 'Culture',
 'WELLNESS': 'Health',
 'PARENTING': 'Other',
 'HOME & LIVING': 'Trends',
 'STYLE & BEAUTY': 'Trends',
 'DIVORCE': 'Other',
 'WEDDINGS': 'Other',
 'FOOD & DRINK': 'Culture',
 'MONEY': 'Other',
 'ENVIRONMENT': 'Activism',
 'CULTURE & ARTS': 'Culture'}

In [7]:
new_data = data.copy()
new_data['category'] = data.category.map(lambda x: my_dict[x])

In [8]:
new_data.head()

,category,headline,authors,link,short_description,date
0,Crime,There Were 2 Mass Shootings In Texas Last Week...,Melissa Jeltsen,https://www.huffingtonpost.com/entry/texas-ama...,She left her husband. He killed their children...,2018-05-26
1,Entertainment,Will Smith Joins Diplo And Nicky Jam For The 2...,Andy McDonald,https://www.huffingtonpost.com/entry/will-smit...,Of course it has a song.,2018-05-26
2,Entertainment,Hugh Grant Marries For The First Time At Age 57,Ron Dicker,https://www.huffingtonpost.com/entry/hugh-gran...,The actor and his longtime girlfriend Anna Ebe...,2018-05-26
3,Entertainment,Jim Carrey Blasts 'Castrato' Adam Schiff And D...,Ron Dicker,https://www.huffingtonpost.com/entry/jim-carre...,The actor gives Dems an ass-kicking for not fi...,2018-05-26
4,Entertainment,Julianna Margulies Uses Donald Trump Poop Bags...,Ron Dicker,https://www.huffingtonpost.com/entry/julianna-...,"The ""Dietland"" actress said using the bags is ...",2018-05-26


In [9]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('distilbert-base-nli-mean-tokens')
embeddings = model.encode(new_data.headline, show_progress_bar=True)
pd.DataFrame(embeddings).to_csv('../big_picture/data/embeddings_200k.csv')

Batches:   0%|          | 0/6277 [00:00<?, ?it/s]